In [ ]:
#検証時、メッシュ確認で利用したクエリ

In [53]:
import psycopg2

In [61]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [55]:
sql = "select kddi_tran.mesh,ktran900,T.mesh,T.ttran900 "
sql += "from kddi_tran "
sql += "INNER JOIN pointdata as a ON kddi_tran.mesh = cast(a.grid_code AS BIGINT) "
sql += "CROSS JOIN LATERAL"
sql += "  ("
sql += "    SELECT mesh,ttran900 from trakan_tran"
sql += "    inner JOIN pointdata as b ON trakan_tran.mesh = cast(grid_code AS BIGINT) "
sql += "    ORDER BY a.geom <-> b.geom, ST_Distance(a.geom, b.geom) "
sql += "    LIMIT 1) AS T"
cur =connection.cursor()
cur.execute(sql)
target_list = []
for row in cur:
    if row[1] >= 0 && row[3] >= 0 :
        target_list.append(row)
cur.close()

KeyboardInterrupt: 

In [ ]:
target_list

In [48]:
cntDt = 0
cur =connection.cursor()
for row in target_list:
    sql = "SELECT kddi_tran.mesh,ktran900 from kddi_tran "
    sql += "WHERE kddi_tran.mesh = '" + str(row[2]) + "'"
    cur.execute(sql)
    hits = 0
    for row in cur:
        target_list[cntDt] += row[1],
        hits += 1
    if hits == 0:
        target_list[cntDt] += -1,
    cntDt += 1
cur.close()


In [ ]:
target_list

In [51]:
result_list = []
for row in target_list:
    if row[4] != -1:
        param = row[1] * row[3] / row[4]
        wktpl = row[0], param
        result_list.append(wktpl)

In [ ]:
result_list

In [62]:
    with connection.cursor() as cursor:
        # レコードを挿入
        sql = "CREATE TABLE margeresult ("
        sql += "mesh BIGINT NULL DEFAULT NULL,"
        sql += "params DOUBLE NULL DEFAULT NULL"
        sql += ");"
        sql += "COMMENT ON COLUMN mesh IS '';"
        sql += "COMMENT ON COLUMN params IS '';"
        cursor.execute(sql)
    # コミットしてトランザクション実行
    connection.commit()

SyntaxError: 列名を修飾する必要があります


In [58]:
for row in target_list:
    with connection.cursor() as cursor:
        # レコードを挿入
        sql = "INSERT INTO margeresult (mesh, params) VALUES ("+str(row[0])+", "+str(row[1])+")"
        cursor.execute(sql)
    # コミットしてトランザクション実行
    connection.commit()

UndefinedTable: リレーション"margeresult"は存在しません
LINE 1: INSERT INTO margeresult (mesh, params) VALUES (51320614132, ...
                    ^


In [ ]:
connection.close()